In [1]:
import time

import matplotlib.pyplot as plt

import osmnx as ox
import folium

from geopy.geocoders import Nominatim

from utilities.utils import *
from optimiser import *
from addressPoint import AddressPoint

In [2]:
address_file_path = './data/addresses.csv'

In [3]:
employee_data = get_employee_data(address_file_path)
employee_addresses = [emp[2] for emp in employee_data]

In [4]:
target_address = '9 Boulevard Decouz, 74000 Annecy France'

In [5]:
max_pass = 4

Speed Test

In [13]:
t_start = time.time()
emp_points = {emp_address: AddressPoint(emp_address) for emp_address in employee_addresses}
t_end = time.time()
full_time = t_end - t_start
print('time: {}'.format(full_time))

time: 54.591328620910645


In [8]:
import asyncio
from geopy.adapters import AioHTTPAdapter
from geopy.geocoders import Nominatim

In [9]:
async def get_latitude_longitude_async(str_address):
    print('retrieving for {}'.format(str_address))
    async with Nominatim(user_agent="covoiturage", adapter_factory=AioHTTPAdapter) as geolocator:
        location = await geolocator.geocode(str_address, timeout=500)
        if location is None:
            res = None, None
        else:
            res = location.latitude, location.longitude
            print('address: {} retrieved'.format(str_address))
    return res

In [10]:
async def main(emp_add):
    ret = await asyncio.gather(*[get_latitude_longitude_async(add) for add in emp_add])
    return ret

t_start = time.time()
res = await main(test)
t_end = time.time()
full_time = t_end - t_start
print('time: {}'.format(full_time))

In [ ]:
res

In [ ]:
t_start = time.time()
emp_points_2 = {}
geolocator = Nominatim(user_agent="covoiturage")

for add in employee_addresses:
    print(add)
    location = geolocator.geocode(add)
    if location is None:
        res = None, None
    else:
        res = location.latitude, location.longitude
    emp_points_2[add] = (add, res[0], res[1])
t_end = time.time()
full_time = t_end - t_start
print('time: {}'.format(full_time))

In [ ]:
add_list = list(emp_points.keys())
pA = add_list[0]
pB = add_list[25]
t_start = time.time()
dAB = get_route_distance(emp_points[pA], emp_points[pB])
t_end = time.time()
full_time = t_end - t_start
print('time: {}'.format(full_time))

# result only

In [ ]:
res = global_optimise(employee_addresses, target_address, max_pass)
res

# step by step

In [ ]:
emp_points = {emp_address: AddressPoint(emp_address) for emp_address in employee_addresses}

In [15]:
tgt_point = AddressPoint(target_address)

In [16]:
emp_coords = {k: cartesian_coordinates(p, tgt_point, tgt_point.latitude)
                  for k, p in emp_points.items()}

In [ ]:
tgt_dist_mat = get_employees_distance_to_target(emp_points, tgt_point)

In [ ]:
emp_dist_mat = get_employees_distance_matrix(emp_points)

In [ ]:
init_groups = group_employees(emp_coords, max_pass)

In [ ]:
fin_groups = []
for g in init_groups:
    fin_groups += optimise_group(g, emp_dist_mat, tgt_dist_mat, emp_coords)
fin_groups

In [ ]:
plt.scatter([0], [0])
for g in fin_groups:
    plt.scatter([emp_coords[add][0] for add in g], [emp_coords[add][1] for add in g])

In [ ]:
worst_dist = sum(list(tgt_dist_mat.values()))
worst_dist

In [ ]:
path_lengths = [path_distance(path, emp_dist_mat, tgt_dist_mat) for path in fin_groups]
path_lengths

In [ ]:
opt_dist = sum(path_lengths)
opt_dist

In [ ]:
full_map = folium.Map(location=[tgt_point.latitude, tgt_point.longitude], zoom_start=11)
folium.Marker([tgt_point.latitude, tgt_point.longitude]).add_to(full_map)
for cur_point in emp_points.values():
    folium.Circle(radius = 10, location=[cur_point.latitude, cur_point.longitude], color='red', fill=True).add_to(full_map)
full_map